In [1]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds

tf.config.list_physical_devices('GPU') 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
DATAPATH = os.path.abspath('../../data/')

1 Physical GPUs, 1 Logical GPUs


In [2]:
# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [3]:
tokenizer = info.features['text'].encoder

In [4]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         41088     
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 573,249
Trainable params: 573,249
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 49s 113ms/step - loss: 0.5896 - accuracy: 0.6536 - val_loss: 0.3172 - val_accuracy: 0.8750
Epoch 2/10
391/391 [==============================] - 20s 51ms/step - loss: 0.2337 - accuracy: 0.9131 - val_loss: 0.3162 - val_accuracy: 0.8735
Epoch 3/10
391/391 [==============================] - 17s 45ms/step - loss: 0.1666 - accuracy: 0.9404 - val_loss: 0.3538 - val_accuracy: 0.8657
Epoch 4/10
391/391 [==============================] - 16s 41ms/step - loss: 0.1302 - accuracy: 0.9555 - val_loss: 0.3733 - val_accuracy: 0.8705
Epoch 5/10
391/391 [==============================] - 16s 40ms/step - loss: 0.1053 - accuracy: 0.9660 - val_loss: 0.4358 - val_accuracy: 0.8650
Epoch 6/10
391/391 [==============================] - 16s 41ms/step - loss: 0.0821 - accuracy: 0.9732 - val_loss: 0.4728 - val_accuracy: 0.8588
Epoch 7/10
391/391 [==============================] - 15s 38ms/step - loss: 0.0642 - accuracy: 0.9803 - val_loss: 0.5986 - val_accuracy

In [9]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()